In [1]:
import numpy as np

INF = 10**6  # da onemogucim self-loop

class Hungarian1:
    def __init__(self, cost_matrix):
        self.cost_matrix = np.array(cost_matrix)        
        self.n = self.cost_matrix.shape[0]                      #broj redova/kolona
        self.size = self.n
        self.marked = np.zeros((self.n, self.n))                # 1 - nezavisna nula, 2 - ostale nule (privremeno oznacim)
        self.row_covered = np.zeros(self.n, dtype=bool)
        self.col_covered = np.zeros(self.n, dtype=bool)

    #oduzimamo min elem po vrstama
    def step1(self):
        for i in range(self.n):
            self.cost_matrix[i] -= self.cost_matrix[i].min()

    #oduzimamo min elem po kolonama
    def step2(self):
        for j in range(self.n):
            self.cost_matrix[:, j] -= self.cost_matrix[:, j].min()


    #inicijalno oznacavanje nazav nula
    def step3(self):
        for i in range(self.n):
            for j in range(self.n):
                if self.cost_matrix[i, j] == 0 and not self.row_covered[i] and not self.col_covered[j]:
                    self.marked[i, j] = 1               #oznaci nezavisnu nulu
                    self.row_covered[i] = True          #oznaci samo da ne uzmes sl nezav nulu odatle    
                    self.col_covered[j] = True          #oznaci samo da ne uzmes sl nezav nulu odatle 
        self.row_covered[:] = False                     #ocistimo prekrivene redove i kolone
        self.col_covered[:] = False


    #provjera kraja - ako je br kolona koje sadrze nezav nulu = n -> kraj
    def cover_columns_with_stars(self):
        for j in range(self.n):
            if np.any(self.marked[:, j] == 1):
                self.col_covered[j] = True

    
    #prva nepokrivena nula
    def find_a_zero(self):
        for i in range(self.n):
            for j in range(self.n):
                if self.cost_matrix[i, j] == 0 and not self.row_covered[i] and not self.col_covered[j]:        
                    return i, j
        return None


    #postoji li oznacena nula u redu (ako postoji vratim kolonu)
    def find_star_in_row(self, row):
        for j in range(self.n):
            if self.marked[row, j] == 1:
                return j
        return None


    #postoji li oznacena nula u koloni (ako postoji vratim red)
    def find_star_in_col(self, col):
        for i in range(self.n):
            if self.marked[i, col] == 1:
                return i
        return None


    #kolona sa neoznacenom nulom
    def find_prime_in_row(self, row):
        for j in range(self.n):
            if self.marked[row, j] == 2:
                return j
        return None

    def step4(self):
        while True:
            z = self.find_a_zero()                                  #trazim nepokrivene nule
            if z is None:
                return                                              #ako su sve pokrivene ispadam
            i, j = z
            self.marked[i, j] = 2                                   #markiraj da je nula (obicna)
            star_col = self.find_star_in_row(i)                     #postoji li oznacena u tom redu
            if star_col is not None:                                #ako u tom redu imamo oznacenu nulu
                self.row_covered[i] = True                          #oznacicemo taj red                         
                self.col_covered[star_col] = False                  #skinucemo tu kolonu, zato sto mozda sad mozemo iskoristiti nule u toj koloni
            else:                                                   #kad nadjemo red bez pokrivene nule
                self.augment_path((i, j))                       
                self.row_covered[:] = False
                self.col_covered[:] = False
                self.marked[self.marked == 2] = 0
                self.cover_columns_with_stars()
                return

    def augment_path(self, z0):
        path = [z0]                                                 #z0 je pocetna nule bez nezavisne u svom redu
        while True:
            row = self.find_star_in_col(path[-1][1])                #path cuva redoslijed koord koje cu mijenjati 1 → 0, 0/2 → 1
            if row is None:
                break
            path.append((row, path[-1][1]))
            col = self.find_prime_in_row(path[-1][0])
            path.append((path[-1][0], col))

        for (i, j) in path:                                         #zamjena nula (prime ce postati nezavisna i obrnuto)
            if self.marked[i, j] == 1:
                self.marked[i, j] = 0
            else:
                self.marked[i, j] = 1

    #promjena vrijednosti u matrici
    def step5(self):
        vals = []
        for i in range(self.n):
            for j in range(self.n):
                if not self.row_covered[i] and not self.col_covered[j]:
                    vals.append(self.cost_matrix[i, j])
        if not vals:
            return
        minval = min(vals)

        for i in range(self.n):
            for j in range(self.n):
                if self.row_covered[i]:
                    self.cost_matrix[i, j] += minval             #ako je na granici dodas, ako je vanka oduzmes
                if not self.col_covered[j]:
                    self.cost_matrix[i, j] -= minval

    def solve(self):
        self.step1()
        self.step2()
        self.step3()
        self.cover_columns_with_stars()

        while not all(self.col_covered):                        #iteracija
            self.step4()
            if all(self.col_covered):
                break
            self.step5()

        result = []
        for i in range(self.n):
            for j in range(self.n):
                if self.marked[i, j] == 1:
                    result.append((i, j))
        return result


def preprocess_matrix(matrix):
    m = [row[:] for row in matrix]
    for i in range(len(m)):
        m[i][i] = INF                   # zabrana za self-loop, da mi na glavnoj bude inf za penalty
    return m


def assignment_to_tours(assignment):
    mapping = {i: j for i, j in assignment}
    tours = []
    visited = set()
    for start in mapping:
        if start not in visited:
            cycle = []
            cur = start
            while cur not in visited:
                visited.add(cur)
                cycle.append(cur)
                cur = mapping[cur]
            tours.append(cycle)
    return tours


def merge_cycles_into_tour(assignment, distance_matrix):
    # Spaja subture heuristički u jednu TSP turu
    tours = assignment_to_tours(assignment)
    while len(tours) > 1:
        # pronadi dve najbliže subture da spojimo
        min_dist = float('inf')
        merge_idx = (0, 1)
        for i in range(len(tours)):
            for j in range(i+1, len(tours)):
                for a in [tours[i][0], tours[i][-1]]:
                    for b in [tours[j][0], tours[j][-1]]:
                        if distance_matrix[a][b] < min_dist:
                            min_dist = distance_matrix[a][b]
                            merge_idx = (i, j)
                            connect = (a, b)
        # spajanje toura i update
        i, j = merge_idx
        t1, t2 = tours[i], tours[j]
        a, b = connect
        if t1[0] == a:
            t1 = t1[::-1]
        if t2[-1] == b:
            t2 = t2[::-1]
        tours[i] = t1 + t2
        tours.pop(j)
    return tours[0]


cities = ["A", "B", "C", "D", "E", "F"] 

distance_matrix = [ [0, 12, 10, 19, 8, 14],
                    [12, 0, 3, 7, 2, 11],
                    [10, 3, 0, 6, 20, 4],
                    [19, 7, 6, 0, 13, 9], 
                    [8, 2, 20, 13, 0, 16], 
                    [14, 11, 4, 9, 16, 0] ]

solver1 = Hungarian1(preprocess_matrix(distance_matrix))
assignment1 = solver1.solve()
merged_tour1 = merge_cycles_into_tour(assignment1, distance_matrix)

print("Optimal assignment for matrix 1:")
for i, j in assignment1:
    print(f"{cities[i]} -> {cities[j]} (cost {distance_matrix[i][j]})")
print("Total cost:", sum(distance_matrix[i][j] for i, j in assignment1))
print("Subtours:", [[cities[x] for x in cycle] for cycle in assignment_to_tours(assignment1)])
print("Merged TSP tour:", [cities[x] for x in merged_tour1])


cities2 = ["A", "B", "C", "D", "E", "F", "G", "H", "I"]

distance_matrix2 = [
    [0, 12, 10, 19, 8, 14, 11, 7, 15],
    [12, 0, 3, 7, 2, 11, 9, 6, 10],
    [10, 3, 0, 6, 20, 4, 8, 12, 5],
    [19, 7, 6, 0, 13, 9, 10, 15, 8],
    [8, 2, 20, 13, 0, 16, 7, 11, 14],
    [14, 11, 4, 9, 16, 0, 12, 8, 10],
    [11, 9, 8, 10, 7, 12, 0, 5, 13],
    [7, 6, 12, 15, 11, 8, 5, 0, 9],
    [15, 10, 5, 8, 14, 10, 13, 9, 0]
]

solver2 = Hungarian1(preprocess_matrix(distance_matrix2))
assignment2 = solver2.solve()
merged_tour2 = merge_cycles_into_tour(assignment2, distance_matrix2)

print("\nOptimal assignment for matrix 2:")
for i, j in assignment2:
    print(f"{cities2[i]} -> {cities2[j]} (cost {distance_matrix2[i][j]})")
print("Total cost:", sum(distance_matrix2[i][j] for i, j in assignment2))
print("Subtours:", [[cities2[x] for x in cycle] for cycle in assignment_to_tours(assignment2)])
print("Merged TSP tour:", [cities2[x] for x in merged_tour2])


Optimal assignment for matrix 1:
A -> E (cost 8)
B -> D (cost 7)
C -> F (cost 4)
D -> B (cost 7)
E -> A (cost 8)
F -> C (cost 4)
Total cost: 38
Subtours: [['A', 'E'], ['B', 'D'], ['C', 'F']]
Merged TSP tour: ['A', 'E', 'B', 'D', 'C', 'F']

Optimal assignment for matrix 2:
A -> H (cost 7)
B -> E (cost 2)
C -> F (cost 4)
D -> I (cost 8)
E -> B (cost 2)
F -> C (cost 4)
G -> A (cost 11)
H -> G (cost 5)
I -> D (cost 8)
Total cost: 51
Subtours: [['A', 'H', 'G'], ['B', 'E'], ['C', 'F'], ['D', 'I']]
Merged TSP tour: ['A', 'H', 'G', 'E', 'B', 'C', 'F', 'D', 'I']


In [2]:
import numpy as np

class Hungarian2:
    def __init__(self, cost_matrix):
        self.cost_matrix = np.array(cost_matrix, dtype=float)
        self.n = self.cost_matrix.shape[0]
        self.marked = np.zeros((self.n, self.n))  # 1 = nezavisna nula
        self.row_covered = np.zeros(self.n, dtype=bool)
        self.col_covered = np.zeros(self.n, dtype=bool)

    # Step1: oduzimanje min po vrstama
    def step1(self):
        for i in range(self.n):
            self.cost_matrix[i] -= self.cost_matrix[i].min()

    # Step2: oduzimanje min po kolonama
    def step2(self):
        for j in range(self.n):
            self.cost_matrix[:, j] -= self.cost_matrix[:, j].min()

    # Step3: inicijalno oznacavanje nezavisnih nula
    def step3(self):
        self.marked[:] = 0
        self.row_covered[:] = False
        self.col_covered[:] = False

        for i in range(self.n):
            for j in range(self.n):
                if self.cost_matrix[i, j] == 0 and not self.row_covered[i] and not self.col_covered[j]:
                    self.marked[i, j] = 1
                    self.row_covered[i] = True
                    self.col_covered[j] = True

        # resetujemo prekrivene redove i kolone (TREBALO MI SAD SAMO DA OZNACIM)
        self.row_covered[:] = False
        self.col_covered[:] = False

    # Iterativno prekrivanje
    def iterativno(self):
        while True:
            # 1) redovi bez nezavisne nule
            rows_without_star = [i for i in range(self.n) if not np.any(self.marked[i] == 1)]

            # 2) kolone koje sadrže nule u tim redovima
            precrtane_kolone = set()
            for row in rows_without_star:
                for j in range(self.n):
                    if self.cost_matrix[row, j] == 0:
                        precrtane_kolone.add(j)

            # 3) redovi koji imaju nezavisne nule u precrtanih kolonama
            oznaceni_redovi_sa_nn = set()
            for col in precrtane_kolone:
                for i in range(self.n):
                    if self.marked[i, col] == 1:
                        oznaceni_redovi_sa_nn.add(i)

            # 4) precrtani redovi = svi redovi koji nisu u oznaceni_redovi_sa_nn
            precrtani_redovi = [i for i in range(self.n) if i not in oznaceni_redovi_sa_nn]

            # update covered arrays
            self.row_covered[:] = False
            self.col_covered[:] = False
            for r in precrtani_redovi:
                self.row_covered[r] = True
            for c in precrtane_kolone:
                self.col_covered[c] = True

            # kraj ako je broj prekrivenih kolona + redova = n
            if np.sum(self.col_covered) + np.sum(self.row_covered) == self.n:
                break

            self.transformacija_matrice()


    # Step5: transformacija matrice
    def transformacija_matrice(self):
        vals = []
        for i in range(self.n):
            for j in range(self.n):
                if not self.row_covered[i] and not self.col_covered[j]:
                    vals.append(self.cost_matrix[i, j])
        if not vals:
            return
        minval = min(vals)

        for i in range(self.n):
            for j in range(self.n):
                if self.row_covered[i]:
                    self.cost_matrix[i, j] += minval
                if not self.col_covered[j]:
                    self.cost_matrix[i, j] -= minval


    def solve(self):
        self.step1()
        self.step2()
        self.step3()
        self.iterativno()

        # vrati listu assignment (i,j)
        result = []
        for i in range(self.n):
            for j in range(self.n):
                if self.marked[i, j] == 1:
                    result.append((i, j))
        return result


def assignment_to_tours2(assignment, n):
    mapping = {i: j for i, j in assignment}
    tours = []
    visited = set()

    for start in range(n):  # prolazimo kroz sve gradove
        if start not in visited:
            cycle = []
            cur = start
            while cur not in visited:
                visited.add(cur)
                cycle.append(cur)
                cur = mapping.get(cur, None)
                if cur is None:
                    break  
            if cycle:
                tours.append(cycle)

    return tours


def merge_cycles_into_tour2(assignment, distance_matrix, n):
    tours = assignment_to_tours2(assignment, n)

    while len(tours) > 1:
        min_dist = float('inf')
        merge_idx = (0, 1)
        for i in range(len(tours)):
            for j in range(i + 1, len(tours)):
                for a in [tours[i][0], tours[i][-1]]:
                    for b in [tours[j][0], tours[j][-1]]:
                        if distance_matrix[a][b] < min_dist:
                            min_dist = distance_matrix[a][b]
                            merge_idx = (i, j)
                            connect = (a, b)

        i, j = merge_idx
        t1, t2 = tours[i], tours[j]
        a, b = connect

        if t1[0] == a:
            t1 = t1[::-1]
        if t2[-1] == b:
            t2 = t2[::-1]

        tours[i] = t1 + t2
        tours.pop(j)

    return tours[0]



if __name__ == "__main__":
    cities = ["A", "B", "C", "D", "E", "F"] 

    distance_matrix = [ [10000, 12, 10, 19, 8, 14],
                        [12, 10000, 3, 7, 2, 11],
                        [10, 3, 10000, 6, 20, 4],
                        [19, 7, 6, 10000, 13, 9], 
                        [8, 2, 20, 13, 10000, 16], 
                        [14, 11, 4, 9, 16, 10000] ]

    solver = Hungarian2(distance_matrix)
    assignment = solver.solve()
    n = len(cities)
    merged_tour = merge_cycles_into_tour2(assignment, distance_matrix, n)

    print("Merged TSP tour:", [cities[x] for x in merged_tour])


Merged TSP tour: ['A', 'E', 'B', 'C', 'F', 'D']
